In [3]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook



In [4]:
# Load API Credentials
with open('/Users/paula/.secret/yelp_api.json') as f:   #use your path here!
    login = json.load(f)
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)



In [10]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = f"data/results_in_progress_Ventura_beer.json"
JSON_FILE



'data/results_in_progress_Ventura_beer.json'

In [13]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    ## INFORM USER AND SAVE EMPTY LIST
    print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
    
    
    ## save an empty list to start the file
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
## If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] data/results_in_progress_Ventura_beer.json already exists.


In [14]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')



- 0 previous results found.


In [15]:
# use our yelp_api variable's search_query method to perform our API call
LOCATION = 'Ventura, CA'
TERM = 'beer'

results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()



dict_keys(['businesses', 'total', 'region'])

In [16]:
## How many results total?
total_results = results['total']
total_results



394

In [17]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page



20

In [18]:
# Import additional packages for controlling our loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages



20

In [19]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)


In [20]:
from tqdm.notebook import tqdm_notebook
import time

for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 


  0%|          | 0/20 [00:00<?, ?it/s]

In [21]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
#     ass="function call from-rainbow">display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    # add a 200ms pause
    time.sleep(.2)



  0%|          | 0/20 [00:00<?, ?it/s]

In [22]:
## delete file and confirm it no longer exits.
os.remove(JSON_FILE)
os.path.isfile(JSON_FILE)



False

In [23]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)  



In [24]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)


## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)

print(f'- {n_results} previous results found.')

# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages



[i] data/results_in_progress_Ventura_beer.json not found. Saving empty list to new file.
- 0 previous results found.


20

In [25]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)



  0%|          | 0/20 [00:00<?, ?it/s]

In [26]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,7-18hDIQaFTxnUjl0xDBHA,ventura-coast-brewing-ventura-2,Ventura Coast Brewing,https://s3-media4.fl.yelpcdn.com/bphoto/a4kfvD...,False,https://www.yelp.com/biz/ventura-coast-brewing...,171,"[{'alias': 'breweries', 'title': 'Breweries'}]",4.5,"{'latitude': 34.27974, 'longitude': -119.29426}",[],$$,"{'address1': '76 South Oak St', 'address2': ''...",+18056678640,(805) 667-8640,5722.699034
1,wWezojVpRxlh4w6-WFgm9Q,transmission-brewing-ventura,Transmission Brewing,https://s3-media1.fl.yelpcdn.com/bphoto/GhNfol...,False,https://www.yelp.com/biz/transmission-brewing-...,86,"[{'alias': 'breweries', 'title': 'Breweries'}]",4.5,"{'latitude': 34.276787, 'longitude': -119.284099}",[],$$,"{'address1': '1098 E Front St', 'address2': No...",+18057514075,(805) 751-4075,4722.700741
2,IGARNTZT-nddNwigaX-Zvg,poseidon-brewing-company-ventura-4,Poseidon Brewing Company,https://s3-media3.fl.yelpcdn.com/bphoto/g_t2d3...,False,https://www.yelp.com/biz/poseidon-brewing-comp...,158,"[{'alias': 'breweries', 'title': 'Breweries'}]",4.5,"{'latitude': 34.2437449, 'longitude': -119.209...",[],$$,"{'address1': '5777 Olivas Park Dr', 'address2'...",+18054770239,(805) 477-0239,3292.309520
3,DV4NN8OuP9usuMa5B7ADpA,seaward-brewing-ventura,Seaward Brewing,https://s3-media4.fl.yelpcdn.com/bphoto/ALOD_T...,False,https://www.yelp.com/biz/seaward-brewing-ventu...,49,"[{'alias': 'breweries', 'title': 'Breweries'}]",5.0,"{'latitude': 34.25559, 'longitude': -119.23179}",[delivery],NaN,"{'address1': '1920 Goodyear Ave', 'address2': ...",+18055077202,(805) 507-7202,1088.087173
4,Ch9YY1-adspJ-HLU-yaP1Q,madewest-brewing-company-ventura-pier-ventura,Madewest Brewing Company - Ventura Pier,https://s3-media1.fl.yelpcdn.com/bphoto/h8LuTG...,False,https://www.yelp.com/biz/madewest-brewing-comp...,62,"[{'alias': 'beerbar', 'title': 'Beer Bar'}]",4.5,"{'latitude': 34.27574067, 'longitude': -119.29...",[],$$,"{'address1': '668 E Harbor Blvd', 'address2': ...",+18059475002,(805) 947-5002,5299.895952


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
389,DeYnJBoxvSgpKCyZPJyqIQ,chesters-asia-restaurant-camarillo-3,Chester's Asia Restaurant,https://s3-media4.fl.yelpcdn.com/bphoto/5F8dRR...,False,https://www.yelp.com/biz/chesters-asia-restaur...,334,"[{'alias': 'chinese', 'title': 'Chinese'}]",3.5,"{'latitude': 34.2221327, 'longitude': -119.040...",[delivery],$$,"{'address1': '2216 Pickwick Dr', 'address2': '...",+18054826564,(805) 482-6564,18536.564175
390,MKYUx1FXTH2XYVHBpw2lYA,tresierras-supermarkets-oxnard,Tresierras Supermarkets,https://s3-media1.fl.yelpcdn.com/bphoto/5zKsYc...,False,https://www.yelp.com/biz/tresierras-supermarke...,21,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",2.5,"{'latitude': 34.154843, 'longitude': -119.1780...",[],$$,"{'address1': '133 W Pleasant Valley Rd', 'addr...",+18054876666,(805) 487-6666,13193.182333
391,oGYOov1su16Y3_qq9-xBfA,rubys-fresh-mexican-food-and-tequila-bar-camar...,Ruby's Fresh Mexican Food and Tequila Bar,https://s3-media3.fl.yelpcdn.com/bphoto/CkjPAU...,False,https://www.yelp.com/biz/rubys-fresh-mexican-f...,243,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",3.0,"{'latitude': 34.2365668717825, 'longitude': -1...","[pickup, delivery]",$$,"{'address1': '2508 Las Posas Rd', 'address2': ...",+18053839100,(805) 383-9100,18436.756347
392,m59QH8AhGdYdgfGeW6gyqQ,sumo-sushi-camarillo,Sumo Sushi,https://s3-media2.fl.yelpcdn.com/bphoto/EhnAJ1...,False,https://www.yelp.com/biz/sumo-sushi-camarillo?...,338,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",3.5,"{'latitude': 34.21885, 'longitude': -119.05168}",[delivery],$$,"{'address1': '1490 E Daily Dr', 'address2': ''...",+18054451001,(805) 445-1001,17593.958289
393,rt9GRhxO2gH_JM1BZFut2A,subway-oxnard-31,Subway,https://s3-media3.fl.yelpcdn.com/bphoto/WxbjLb...,False,https://www.yelp.com/biz/subway-oxnard-31?adju...,17,"[{'alias': 'sandwiches', 'title': 'Sandwiches'}]",2.5,"{'latitude': 34.2032474961078, 'longitude': -1...","[pickup, delivery]",$,"{'address1': '200 Del Norte Blvd', 'address2':...",+18054856220,(805) 485-6220,12096.959118


### check for duplicate results
`final_df.duplicated().sum()`

Because our yelp results include columns that contain lists, we cannot check every column in dataframe for duplicates.
Instead, we can use the subset argument for df.duplicated() and df.drop_duplicates() to only check the id column for duplicates.



In [28]:
# check for duplicate ID's 
final_df.duplicated(subset='id').sum()



0

In [29]:
## Drop duplicate ids and confirm there are no more duplicates
final_df = final_df.drop_duplicates(subset='id')
final_df.duplicated(subset='id').sum()


0

In [30]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_Ventura_beer.csv.gz', compression='gzip',index=False)

